In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# We can view all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
measurement_ref = Base.classes.measurement
station_ref = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results

#Find the last date in the database
last_date = session.query(measurement_ref.date).order_by(measurement_ref.date.desc()).first()
last_date[0]

In [ ]:
# Calculate the date 1 year ago (i.e., 12 months)

query_date = dt.date(2017, 8, 23) - dt.timedelta(weeks=52)
query_date

In [ ]:
# Query for date and precipitation

last_12_months = session.query(measurement_ref.date, measurement_ref.prcp).\
    filter(measurement_ref.date >= query_date).all()

In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date column

precip_df = pd.DataFrame(last_12_months)
precip_df = precip_df.set_index("date")
precip_df

In [ ]:
# Sort the dataframe by date
precip_df = precip_df.sort_values("date")
precip_df

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
precip_df.plot(figsize=(12, 6))
plt.xlabel("Date")
plt.ylabel("Precipitation (in)")
plt.title("Honolulu, Hawaii Precipitation \n Last 12 Months")
plt.savefig("images/hon_hawaii_precip_timeseries.jpeg")

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data

summary_precip_df = precip_df.groupby("date").agg({"prcp": ["mean", "median", "var", "std", "sem"]})
summary_precip_df

In [ ]:
# Design a query to show how many stations are available in this dataset?
count_stations = session.query(func.count(measurement_ref.station)).group_by(measurement_ref.station).count()
print(f"There are {count_stations} unique station IDs.")

In [ ]:
count_total_stations_occur = session.query(func.count(measurement_ref.station))
print(f"There are {count_total_stations_occur[0][0]} total occurances of station IDs.")

In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?

grouped_station_count = session.query(func.count(measurement_ref.station)).group_by(measurement_ref.station).all()
grouped_station_count

In [ ]:
# List the stations and the counts in descending order.
ordered_stations = session.query(measurement_ref.station, func.count(measurement_ref.station)).order_by(func.count(measurement_ref.station).desc()).group_by(measurement_ref.station).all()

station_list = []

for station in ordered_stations:
    print(station[0], station[1])

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded
min_temp = session.query(func.min(measurement_ref.tobs)).all()

for temp in min_temp:
    print(f"The lowest temperature recorded was {temp[0]}.")

In [ ]:
# highest temperature recorded, and average temperature of the most active station?
max_temp = session.query(func.max(measurement_ref.tobs)).all()

for temp in max_temp:
    print(f"The highest temperature recorded was {temp[0]}.")

In [ ]:
# Choose the station with the highest number of temperature observations.
all_temp_records = session.query(measurement_ref.station, func.count(measurement_ref.tobs)).\
                    order_by(func.count(measurement_ref.tobs).desc()).group_by(measurement_ref.station).all()

all_temp_records

In [ ]:
most_temp_recs = session.query(measurement_ref.date, measurement_ref.tobs).filter(measurement_ref.station == 'USC00519281').all()
most_temp_recs

In [ ]:
station_USC00519281_df = pd.DataFrame(most_temp_recs)

In [ ]:
station_USC00519281_df = station_USC00519281_df.set_index("date")

In [ ]:
station_USC00519281_df

In [ ]:
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
station_USC00519281_df.plot(kind="hist")
plt.title("Station USC00519281 Temperature Recordings \n 2010 - 2017")
plt.xlabel("Temperature (F)")
plt.ylabel("Number of Temperature Recordings")
plt.savefig("images/station_USC00519281_temp_recordings.jpeg")

## Bonus Challenge Assignment

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
